In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
from itertools import product
from hyperopt import fmin, tpe, hp
from functools import partial
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

import config as cfg

In [26]:
# dates_range = [('2023-01-02', '2023-06-23')]
# watch_days_range = [2]
# hold_days_range = [2]
# num_stocks_to_buy_range = [1]
# loss_limit_range = [0.999]
# fee_range = [0.002]
# ranks = ['nsmallest']
# pct_change_thresholds = [-99, 0.01]

dates_range = [('2000-08-01', '2022-12-31'), ('2000-08-02', '2022-12-31'), ('2000-08-03', '2022-12-31'), ('2000-08-04', '2022-12-31'), ('2000-08-05', '2022-12-31'), ('2000-08-06', '2022-12-31'), ('2000-08-07', '2022-12-31'), ('2000-08-08', '2022-12-31'), ('2000-08-09', '2022-12-31'), ('2000-08-10', '2022-12-31'), ('2000-08-11', '2022-12-31'), ('2000-08-12', '2022-12-31'), ('2000-08-13', '2022-12-31'), ('2000-08-14', '2022-12-31'),
               ('2001-01-01', '2001-12-31'), ('2001-01-02', '2001-12-31'), ('2001-01-03', '2001-12-31'), ('2001-01-04', '2001-12-31'), ('2001-01-05', '2001-12-31'),
               ('2002-01-01', '2002-12-31'), ('2002-01-02', '2002-12-31'), ('2002-01-03', '2002-12-31'), ('2002-01-04', '2002-12-31'), ('2002-01-07', '2002-12-31'),
               ('2003-01-01', '2003-12-31'), ('2003-01-02', '2003-12-31'), ('2003-01-03', '2003-12-31'), ('2003-01-06', '2003-12-31'), ('2003-01-07', '2003-12-31'),
               ('2004-01-01', '2004-12-31'), ('2004-01-02', '2004-12-31'), ('2004-01-05', '2004-12-31'), ('2004-01-06', '2004-12-31'), ('2004-01-07', '2004-12-31'),
               ('2005-01-03', '2005-12-31'), ('2005-01-04', '2005-12-31'), ('2005-01-05', '2005-12-31'), ('2005-01-06', '2005-12-31'), ('2005-01-07', '2005-12-31'),
               ('2006-01-02', '2006-12-31'), ('2006-01-03', '2006-12-31'), ('2006-01-04', '2006-12-31'), ('2006-01-05', '2006-12-31'), ('2006-01-06', '2006-12-31'),
               ('2007-01-01', '2007-12-31'), ('2007-01-02', '2007-12-31'), ('2007-01-03', '2007-12-31'), ('2007-01-04', '2007-12-31'), ('2007-01-05', '2007-12-31'),
               ('2008-01-01', '2008-12-31'), ('2008-01-02', '2008-12-31'), ('2008-01-03', '2008-12-31'), ('2008-01-04', '2008-12-31'), ('2008-01-07', '2008-12-31'),
               ('2009-01-01', '2009-12-31'), ('2009-01-02', '2009-12-31'), ('2009-01-05', '2009-12-31'), ('2009-01-06', '2009-12-31'), ('2009-01-07', '2009-12-31'),
               ('2010-01-01', '2010-12-31'), ('2010-01-04', '2010-12-31'), ('2010-01-05', '2010-12-31'), ('2010-01-06', '2010-12-31'), ('2010-01-07', '2010-12-31'),
               ('2011-01-03', '2011-12-31'), ('2011-01-04', '2011-12-31'), ('2011-01-05', '2011-12-31'), ('2011-01-06', '2011-12-31'), ('2011-01-07', '2011-12-31'),
               ('2012-01-02', '2012-12-31'), ('2012-01-03', '2012-12-31'), ('2012-01-04', '2012-12-31'), ('2012-01-05', '2012-12-31'), ('2012-01-06', '2012-12-31'),
               ('2013-01-01', '2013-12-31'), ('2013-01-02', '2013-12-31'), ('2013-01-03', '2013-12-31'), ('2013-01-04', '2013-12-31'), ('2013-01-07', '2013-12-31'),
               ('2014-01-01', '2014-12-31'), ('2014-01-02', '2014-12-31'), ('2014-01-03', '2014-12-31'), ('2014-01-06', '2014-12-31'), ('2014-01-07', '2014-12-31'),
               ('2015-01-01', '2015-12-31'), ('2015-01-05', '2015-12-31'), ('2015-01-06', '2015-12-31'), ('2015-01-07', '2015-12-31'), ('2015-01-08', '2015-12-31'),
               ('2016-01-01', '2016-12-31'), ('2016-01-04', '2016-12-31'), ('2016-01-05', '2016-12-31'), ('2016-01-06', '2016-12-31'), ('2016-01-07', '2016-12-31'),
               ('2017-01-02', '2017-12-31'), ('2017-01-03', '2017-12-31'), ('2017-01-04', '2017-12-31'), ('2017-01-05', '2017-12-31'), ('2017-01-06', '2017-12-31'),
               ('2018-01-01', '2018-12-31'), ('2018-01-02', '2018-12-31'), ('2018-01-03', '2018-12-31'), ('2018-01-04', '2018-12-31'), ('2018-01-05', '2018-12-31'),
               ('2019-01-01', '2019-12-31'), ('2019-01-02', '2019-12-31'), ('2019-01-03', '2019-12-31'), ('2019-01-04', '2019-12-31'), ('2019-01-07', '2019-12-31'),
               ('2020-01-01', '2020-12-31'), ('2020-01-02', '2020-12-31'), ('2020-01-03', '2020-12-31'), ('2020-01-06', '2020-12-31'), ('2020-01-07', '2020-12-31'),
               ('2021-01-01', '2021-12-31'), ('2021-01-04', '2021-12-31'), ('2021-01-05', '2021-12-31'), ('2021-01-06', '2021-12-31'), ('2021-01-07', '2021-12-31'),
               ('2022-01-03', '2022-12-31'), ('2022-01-04', '2022-12-31'), ('2022-01-05', '2022-12-31'), ('2022-01-06', '2022-12-31'), ('2022-01-07', '2022-12-31'),
               ('2023-01-02', '2023-06-23'), ('2023-01-03', '2023-06-23'), ('2023-01-04', '2023-06-23'), ('2023-01-05', '2023-12-23'), ('2023-01-06', '2023-12-23'),]
watch_days_range = [1,2,3,4,5,6,7]
hold_days_range = [1,2,3,4,5,6,7,10]
num_stocks_to_buy_range = [1,2,3,4]
loss_limit_range = [0.999,0.998,0.997]
fee_range = [0.002, 0.003, 0.006]
ranks = ['nsmallest', 'nlargest']
pct_change_thresholds = [-99, 0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1, 1.5, 2, 3, 5]

tickers = cfg.tickers
num_tickers = len(tickers)

num_combinations = len(dates_range) * len(ranks) * len(pct_change_thresholds) * len(watch_days_range) * len(hold_days_range) * len(num_stocks_to_buy_range) * len(loss_limit_range) * len(fee_range)
combinations_per_minute = 10

print(f"Number of combinations: {num_combinations}, Estimated time: {(num_combinations / combinations_per_minute):.2f} minutes")

Number of combinations: 6761664, Estimated time: 676166.40 minutes


In [27]:
def get_number_of_years(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end - start
    return delta.days / 365.25

In [28]:
def download_stock_data(start_date, end_date):
    stock_data = yf.download(tickers, start=start_date, end=end_date)

    df = pd.DataFrame(stock_data)
    df = df.sort_values("Date") # stock_data is not ordered by default
    
    close_df = df[["Close"]]
    close_df.columns = close_df.columns.droplevel(0) # Convert to single level columns
    
    low_df = df[["Low"]]
    low_df.columns = low_df.columns.droplevel(0)

    return close_df, low_df

[****                   9%                       ]  19 of 208 completed

[*****                 10%                       ]  20 of 208 completed

In [29]:
def get_top_stocks(rank, pct_change_threshold, price_increase, num_stocks_to_buy):
    if rank == 'nlargest':    
        filtered_columns = price_increase.columns[(price_increase > pct_change_threshold).any()].tolist()
        top_stocks = sorted(filtered_columns, key=lambda col: price_increase[col].values[0], reverse=True)[:num_stocks_to_buy]
        return top_stocks

    elif rank == 'nsmallest':
        filtered_columns = price_increase.columns[(price_increase > pct_change_threshold).any()].tolist()
        top_stocks = sorted(filtered_columns, key=lambda col: price_increase[col].values[0], reverse=False)[:num_stocks_to_buy]
        return top_stocks
    
    return None

In [30]:
def calculate_profit(close_df, low_df, buy_day_number, sell_day_number, top_stocks, loss_limit, fee):
    buy_prices = close_df.loc[close_df.index[buy_day_number], top_stocks] # Calculate the buying prices at the start of the hold period
    buy_prices = buy_prices.sort_index(axis=0)

    hold_prices = close_df.loc[close_df.index[sell_day_number], top_stocks] # Calculate the selling prices after watch_days + hold_days days
    hold_prices = hold_prices.sort_index(axis=0)

    limit_prices = loss_limit * buy_prices

    min_prices = low_df.iloc[buy_day_number+1:sell_day_number+1].min() # Calculate the minimum price in the holding period
    min_prices = min_prices.loc[min_prices.index.intersection(top_stocks)]
    min_prices = min_prices.sort_index(axis=0)
    
    condition = min_prices < limit_prices

    sell_prices = hold_prices.copy()
    sell_prices[condition] = limit_prices[condition]

    profits = (sell_prices * (1 - fee)) / (buy_prices * (1 + fee))
    profits = profits.dropna()

    if num_combinations == 1:
        print(f'buy_price: {buy_prices}, sell_prices: {sell_prices}')

    profit = profits.mean(skipna=True) # Calculate the average of profits of the selected stocks

    return profit

def calculate_profit_with_hyperparams(params, close_df, low_df, years):
    rank = params['rank']
    pct_change_threshold = params['pct_change_threshold']
    watch_days = params['watch_days']
    hold_days = params['hold_days']
    num_stocks_to_buy = params['num_stocks_to_buy']
    loss_limit = params['loss_limit']
    fee = params['fee']

    num_iterations = int(len(close_df) / (watch_days + hold_days)) - 1
    total_profits = []

    for i in range(num_iterations):
        start_watch_day_number = i * (watch_days + hold_days)
        buy_day_number = start_watch_day_number + watch_days
        sell_day_number = buy_day_number + hold_days

        price_increase = close_df[start_watch_day_number:buy_day_number].pct_change(watch_days - 1).tail(1)

        if len(price_increase.columns) >= num_stocks_to_buy:
            top_stocks = get_top_stocks(rank, pct_change_threshold, price_increase, num_stocks_to_buy)
            profit = calculate_profit(close_df, low_df, buy_day_number, sell_day_number, top_stocks, loss_limit, fee) if top_stocks is not None else 1
            total_profits.append(profit if np.isfinite(profit) else 1)

    if len(total_profits) > 0:
        if years == 0:
            return 0
        average_yearly_profit = pow(np.prod(total_profits), 1 / years)
        return -average_yearly_profit  # Negative average_yearly_profit to maximize it

    return 0

    return -1

In [31]:
def plot_profits(profits_list, total_profits_list):
    mpl.rcParams['figure.figsize'] = [20, 15]
    mpl.rcParams['axes.grid'] = True

    plt.plot(profits_list)
    plt.title('profit vs iteration')
    plt.show()

    plt.plot(total_profits_list)
    plt.title('total profit vs iteration')
    plt.show()

[******                12%                       ]  24 of 208 completed

In [32]:
def get_win_ratio(profits):
    count = sum(1 for num in profits if num > 1)
    win_ratio = count / len(profits)
    return round(win_ratio, 2)

In [33]:
# def get_results():
#     results = []
#     combination_index = 0

#     for start_date, end_date in dates_range:
#         close_df, low_df = download_stock_data(start_date, end_date)
#         years = get_number_of_years(start_date, end_date)

#         for rank, pct_change_threshold, watch_days, hold_days, num_stocks_to_buy, loss_limit, fee in product(
#             ranks, pct_change_thresholds, watch_days_range, hold_days_range, num_stocks_to_buy_range, loss_limit_range, fee_range
#         ):
#             num_iterations = int(len(close_df) / (watch_days + hold_days)) - 1
#             total_profit = 1
#             profits_list = []
#             total_profits_list = []

#             for i in range(num_iterations):
#                 start_watch_day_number = i * (watch_days + hold_days)
#                 buy_day_number = start_watch_day_number + watch_days
#                 sell_day_number = buy_day_number + hold_days

#                 price_increase = close_df[start_watch_day_number:buy_day_number].pct_change(watch_days - 1).tail(1)

#                 if len(price_increase.columns) >= num_stocks_to_buy:
#                     top_stocks = get_top_stocks(rank, pct_change_threshold, price_increase, num_stocks_to_buy)
#                     profit = calculate_profit(close_df, low_df, buy_day_number, sell_day_number, top_stocks, loss_limit, fee) if top_stocks is not None else 1
#                     total_profit *= profit if np.isfinite(profit) else 1
#                     yearly_profit = pow(total_profit, 1 / years)

#                     profits_list.append(profit)
#                     total_profits_list.append(total_profit)

#             win_ratio = get_win_ratio(profits_list)

#             results.append({
#                 'date_range': f'{start_date}/{end_date}',
#                 'rank': rank,
#                 'pct_change_threshold': pct_change_threshold,
#                 'watch_days': watch_days,
#                 'hold_days': hold_days,
#                 'num_stocks_to_buy': num_stocks_to_buy,
#                 'loss_limit': loss_limit,
#                 'fee': fee,
#                 'num_tickers': num_tickers,
#                 'total_profit': round(total_profit, 4),
#                 'yearly_profit': round(yearly_profit, 4),
#                 'win_ratio': win_ratio
#             })

#             print(f"Combination: {combination_index + 1} / {num_combinations}")
#             combination_index += 1
    
#     if num_combinations == 1:
#         plot_profits(profits_list, total_profits_list)

#     return results

[******                12%                       ]  25 of 208 completed

In [34]:
def add_year_column(df):
    df['year'] = df['date_range'].str[:4]
    column = df.pop("year") # Remove the "year" column from its current position
    df.insert(1, "year", column) # Insert the "year" column at the desired position (index 1)
    
    return df

[******                13%                       ]  27 of 208 completed

In [35]:
def get_cross_tab(df):
    cross_tab = pd.crosstab(
        index=[df['rank'], df['pct_change_threshold'], df['watch_days'], df['hold_days'], df['num_stocks_to_buy'], df['loss_limit'], df['fee'], df['num_tickers']],
        columns=df['year'],
        values=df['yearly_profit'],
        aggfunc=['median', 'mean', 'min', 'max', 'count'],
        margins=True)
    cross_tab = cross_tab.sort_values(by=[('median', 'All')], ascending=False)
    print(cross_tab.to_markdown())
    
    return cross_tab

In [36]:
def get_results_hyperopt(dates_range, tickers, num_tickers):
    space = {
        'rank': hp.choice('rank', ranks),
        'pct_change_threshold': hp.choice('pct_change_threshold', pct_change_thresholds),
        'watch_days': hp.choice('watch_days', watch_days_range),
        'hold_days': hp.choice('hold_days', hold_days_range),
        'num_stocks_to_buy': hp.choice('num_stocks_to_buy', num_stocks_to_buy_range),
        'loss_limit': hp.choice('loss_limit', loss_limit_range),
        'fee': hp.choice('fee', fee_range),
    }

    results = []

    for start_date, end_date in dates_range:
        close_df, low_df = download_stock_data(start_date, end_date)
        years = get_number_of_years(start_date, end_date)

        objective = partial(calculate_profit_with_hyperparams, close_df=close_df, low_df=low_df, years=years)
        best_hyperparams = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

        rank = ranks[best_hyperparams['rank']]
        pct_change_threshold = pct_change_thresholds[best_hyperparams['pct_change_threshold']]
        watch_days = watch_days_range[best_hyperparams['watch_days']]
        hold_days = hold_days_range[best_hyperparams['hold_days']]
        num_stocks_to_buy = num_stocks_to_buy_range[best_hyperparams['num_stocks_to_buy']]
        loss_limit = loss_limit_range[best_hyperparams['loss_limit']]
        fee = fee_range[best_hyperparams['fee']]

        total_profit = calculate_profit_with_hyperparams(best_hyperparams, close_df=close_df, low_df=low_df, years=years)
        yearly_profit = pow(total_profit, 1 / years)

        results.append({
            'date_range': f'{start_date}/{end_date}',
            'rank': rank,
            'pct_change_threshold': pct_change_threshold,
            'watch_days': watch_days,
            'hold_days': hold_days,
            'num_stocks_to_buy': num_stocks_to_buy,
            'loss_limit': loss_limit,
            'fee': fee,
            'num_tickers': num_tickers,
            'total_profit': round(total_profit, 4),
            'yearly_profit': round(yearly_profit, 4),
        })

    results_df = pd.DataFrame(results)
    results_df = add_year_column(results_df)
    results_df.sort_values("yearly_profit", ascending=False, inplace=True)

    return results_df

# Define the list of tickers
tickers = cfg.tickers
num_tickers = len(tickers)

# Call the function to get the results using hyperopt optimization
results_hyperopt_df = get_results_hyperopt(dates_range, tickers, num_tickers)

# Save the results to a CSV file
formatted_datetime = datetime.now().strftime("%Y-%m-%d_%Hh%Mm%Ss")
results_hyperopt_df.to_csv(f'./outputs/results_hyperopt_{formatted_datetime}.csv', index=False)

cross_tab = get_cross_tab(results_hyperopt_df)
cross_tab.to_csv(f'./outputs/pivot_table_hyperopt_{formatted_datetime}.csv')

[*                      2%                       ]  4 of 208 completed

KeyboardInterrupt: 

[****                   9%                       ]  18 of 208 completed

[*******               14%                       ]  29 of 208 completed

[*********************100%***********************]  208 of 208 completeded

7 Failed downloads:
- HOLN.PA: No data found, symbol may be delisted
- COX.PA: No data found, symbol may be delisted
- EDF.PA: No data found, symbol may be delisted
- NAVYA.PA: No data found, symbol may be delisted
- PHA.PA: No data found, symbol may be delisted
- URW.PA: Data doesn't exist for startDate = 965080800, endDate = 1672441200
- EUCAR.PA: No data found, symbol may be delisted
100%|██████████| 100/100 [12:28<00:00,  7.49s/trial, best loss: -1.247004717647041]


/var/folders/1x/b84w4ts97b728gp2r8cchfgw0000gn/T/ipykernel_7318/4237559700.py:30: RuntimeWarning: invalid value encountered in double_scalars
  yearly_profit = pow(total_profit, 1 / years)


[*********************100%***********************]  208 of 208 completed

7 Failed downloads:
- HOLN.PA: No data found, symbol may be delisted
- COX.PA: No data found, symbol may be delisted
- EDF.PA: No data found, symbol may be delisted
- NAVYA.PA: No data found, symbol may be delisted
- PHA.PA: No data found, symbol may be delisted
- URW.PA: Data doesn't exist for startDate = 965167200, endDate = 1672441200
- EUCAR.PA: No data found, symbol may be delisted
 68%|██████▊   | 68/100 [09:18<04:22,  8.21s/trial, best loss: -1.275219667319068] 


KeyboardInterrupt: 